In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
#from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
from pathlib import Path
plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# used for Alexnet
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
valid_size = 0.2
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_train)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_test)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
# define samplers for obtaining training and validation batches
#train_sampler = SubsetRandomSampler(train_idx)
#valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
training_id = 1
model_short_name = 'vgg'
framework = 'pytorch'

In [6]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_exp_vgg_2021-11-03_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

515298923

In [7]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [8]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/errors/'

In [9]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
def model_scores(y_test, test_predict):
    correct_ = np.sum(y_test == test_predict)
    accuracy  = correct_*100./np.sum(y_test == y_test)
    return accuracy

In [10]:
def to_onnx(i, x, y, data_writer_run, batch_size):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    traced_model = torch.jit.trace(model, x)
    
    ### Original Model
    since_1 = time.time()
    torch_out = model(x)
    inference_time_original = time.time() - since_1
    y0 = torch.argmax(torch_out,1)
    accuracy_original = model_scores(to_numpy(y), to_numpy(y0))
    # ONNX Model
    
    t_elapsed_2 = time.time() - since_1
    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    onnx.checker.check_model(onnx_model)
    size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    ort_outs_tensor = torch.from_numpy(ort_outs[0]) 
    y2 = torch.argmax(ort_outs_tensor,1)
    #y2 = to_categorical(np.argmax(ort_outs[0], 1), num_classes = 10)
    #correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(to_numpy(y), to_numpy(y2))
    miss_perc_val_original_runtime = 0
    try:
        np.testing.assert_array_equal(to_numpy(torch_out), ort_outs[0])
    except Exception as e:
        #print(e)
        flag = 0
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                miss_perc_val_original_runtime = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                flag += 1
                break
        if flag == 0:
            print('Error onnx0: ', e)
    encoded_miss_perc_val_original_onnx = 0
    try:
        np.testing.assert_array_equal(to_numpy(y0), to_numpy(y2))
    except Exception as e:
        #print(e)
        flag = 0
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                encoded_miss_perc_val_original_onnx = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                flag += 1
                break
        if flag == 0:
            print('Error onnx1: ', e)
    
    miss_perc_val_test_runtime = 0
    try:
        np.testing.assert_array_equal(to_numpy(y), to_numpy(y2))
    except Exception as e:
        #print(e)
        flag = 0
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                miss_perc_val_test_runtime = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                flag += 1
                break
        if flag == 0:
            print('Error onnx2: ', e)
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    ## Converting the coremltool
    since_1 = time.time()
    # Using image_input in the inputs parameter:
    # Convert to Core ML using the Unified Conversion API.
    coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])
    t_conversion_time_coreml = time.time() - since_1
    since_1 = time.time()
    coreml_model.save(coreml_path+framework+'/{}/{}-v2.mlmodel'.format(model_short_name, model_name))
    t_saving_time_coreml = time.time() - since_1
    
    
    #print(name_1)

    size_coreml = os.path.getsize(coreml_path+framework+'/{}/{}-v2.mlmodel'.format(model_short_name, model_name))
    
    
    since_1 = time.time()
    coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}-v2.mlmodel".format(model_short_name, model_name))
    load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    name_1 = coreml_model.get_spec().description.input[0].name
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    coreml_array_output = output_dict_test[list(output_dict_test.keys())[0]]
    output_tensor = torch.from_numpy(coreml_array_output) 
    y3 = torch.argmax(output_tensor,1)
    #correct_coreml = np.sum(to_numpy(y3) == to_numpy(y))
    accuracy_coreml = model_scores(to_numpy(y3), to_numpy(y))
    
    #print(correct_original, correct_coreml, correct_onnx, np.sum(y == y))
    ## Part 1
    
    miss_perc_val_original_runtime2 = 0
    try:
        np.testing.assert_array_equal(to_numpy(torch_out), coreml_array_output)
    except Exception as e:
        #print(e)
        flag = 0
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                miss_perc_val_original_runtime2 = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                flag += 1
                break
        if flag == 0:
            print('Error coreml0: ', e)
    
    ####### Part 2
    #print('default-shape: ',k_predict.shape, 'onnx-shape: ',ort_outs[0].shape, 'coreml-shape: ',output_dict_test['Identity'].shape)
    miss_perc_val_original_coreml = 0
    try:
        np.testing.assert_array_equal(to_numpy(y0), to_numpy(y3))
    except Exception as e:
        #print(e)
        flag = 0
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                miss_perc_val_original_coreml = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                flag += 1
                break
        if flag == 0:
            print('Error coreml1: ', e)
    miss_perc_val_test_runtime2 = 0
    try:
        np.testing.assert_array_equal(to_numpy(y), to_numpy(y3))
    except Exception as e:
        #print(e)
        flag = 0
        for line_ in str(e).split('\n'):
            #print(' ---- : ', line_)
            if 'Mismatched elements' in line_:
                value = line_.replace('Mismatched elements: ', '').strip()
                miss_perc_val_test_runtime2 = value[value.find("(")+1:value.find(")")]
                #print(value, perc_val)
                flag += 1
                break
        if flag == 0:
            print('Error coreml2: ', e)
    ####### End of mis-classification coreML ######################################
    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'onnx',size0, size_onnx,t_elapsed_0, inference_time_original,'', '', load_time_onnx, 
                          inference_time_onnx,  miss_perc_val_original_runtime,'',  encoded_miss_perc_val_original_onnx, miss_perc_val_test_runtime, '', accuracy_original, accuracy_onnx])


    data_writer_run.writerow([model_short_name, framework, training_id, model_name, batch_size, i,'coremltools',size0, size_coreml,t_elapsed_0, inference_time_original,t_conversion_time_coreml, t_saving_time_coreml, load_time_coreml, 
                          inference_time_coreml,  miss_perc_val_original_runtime2,'',  miss_perc_val_original_coreml, miss_perc_val_test_runtime2, '', accuracy_original, accuracy_coreml])

In [11]:
def _lets_convert(data,  data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    for i, (images, labels) in enumerate(data):
        to_onnx(i, images,labels, data_writer_run, batch_size)
        if i == 50: 
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )

In [12]:
import pandas as pd 
if not os.path.exists(error_path+framework+"/{}".format(model_short_name)):
        Path(error_path+framework+"/{}".format(model_short_name)).mkdir(parents=True, exist_ok=True)
data_file_run = open(error_path+framework+"/{}/runtime_miss-classification_{}.csv".format(model_short_name,model_name), mode='w', newline='',
                                  encoding='utf-8')
data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','runtime','original_memory_size', 'runtime_memory_size','original_load_time', 'original_infererence_time','runtime_conversion_time', 'runtime_saving_time', 'runtime_load_time', 
                          'runtime_inference_time',  'miss_classified_original_runtime_percentage','',  'encoded_miss_classified_original_runtime_percentage','encoded_miss_classified_original_test_runtime_percentage', '', 'accuracy_original', 'accuracy_runtime'])

for batch_size in [128]:
    print("################ Batch size: ", batch_size)
    valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
    _lets_convert(valloader, data_writer_run, batch_size)
    #data_writer_acc.writerow([model_short_name,framework, training_id, model_name, batch_size, correct_original, correct_onnx, correct_coreml])
data_file_run.close()

################ Batch size:  128
converting for batch:  0


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/pytorch/vgg/torch_exp_vgg_2021-11-03_1.onnx'